In [1]:
import sys
import os
import IPython
os.environ['NUMBA_CACHE_DIR'] = IPython.paths.get_ipython_cache_dir() # librosa 충돌 방지
sys.path.append(os.path.join(os.path.dirname(""), "..")) # custom import 용도
import custom
import numpy as np 

from transformers import AutoProcessor, AutoModelForCTC # hugging face 전처리 함수, 신경망 함수
import json # 음절-라벨 사전
import librosa # 소리 파일 불러오기

import torch # torch.tensor 형태로 전처리, 신경망에서 출력 받아서 데이터 처리

In [2]:
# 전처리 함수 불러오기
load_directory = './STT/'

tokenizer = AutoProcessor.from_pretrained(load_directory)

In [3]:
# 신경망 함수 ONNX로 불러와서 저장하기
from optimum.onnxruntime import ORTModelForCTC as ORT

os.makedirs("onnx", exist_ok=True)
save_directory = "./onnx/"

model = ORT.from_pretrained(load_directory, export=True)
model.save_pretrained(save_directory)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
C:\ProgramData\anaconda3\Lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:839: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_output.size() != (bsz, self.num_heads, tgt_len, self.head_dim):


In [4]:
# onnx 불러오기
import onnxruntime

model = onnxruntime.InferenceSession("onnx/model.onnx", providers=["CPUExecutionProvider"])

In [5]:
# 음절-라벨 사전 불러오기
with open("STT/vocab.json", mode = "r", encoding = "UTF8") as f:
    sound_dict = json.load(f)
    num_dict = {v : k for k, v in sound_dict.items()}

print(len(sound_dict))

1452


In [6]:
# 테스트용 소리 파일 불러오기

file = "test.wav"

sig, freq = librosa.load(file, sr = 16000)
print(sig.shape)
print(sig.dtype)
print(freq)
print(sig.min(), sig.max())
print()
print(sig)

(52302,)
float32
16000
-0.38356403 0.33580145

[0. 0. 0. ... 0. 0. 0.]


In [7]:
# hugging face 전처리 함수

data = tokenizer(sig, sampling_rate=freq, return_tensors="np").input_values

print(data)
print(data.shape)

[[-0.00025988 -0.00025988 -0.00025988 ... -0.00025988 -0.00025988
  -0.00025988]]
(1, 52302)


In [8]:
# onnx 신경망 함수

inputs = {model.get_inputs()[0].name : data}
outs = model.run(None, inputs)
y = outs[0]

print(y)
print(y.shape)

[[[  2.769337    -0.68285066  -5.853647   ...  16.073282   -11.053328
   -11.028539  ]
  [  2.5907173   -0.844246    -5.6037292  ...  19.421173   -12.814463
   -12.8727    ]
  [  2.7734795   -0.90178263  -5.6587214  ...  20.029228   -13.323929
   -13.375254  ]
  ...
  [  0.8001082   -1.4135631   -6.508258   ...  21.919247   -13.695276
   -13.874016  ]
  [  5.0337906   -0.8160259   -7.1028633  ...  22.983784   -15.232948
   -15.400426  ]
  [ 20.932737     0.91462773  -2.1320825  ...   7.1835904   -5.4667106
    -5.3573294 ]]]
(1, 163, 1454)


In [9]:
# 결과값 해석

argmax = np.argmax(y, axis = -1).squeeze().tolist() # numpy.array를 list로 변환

text = custom.word_vectorize(argmax, num_dict)
text = "".join(text).replace("[PAD]","").replace("|"," ")

print(text)

디법프 
